This code is for raw json data reading and some funcitons 

In [ ]:
import pandas as pd
import numpy as np
import joblib 
from tqdm import tqdm

#import keras

In [ ]:
dataPath = './trainData/tweets_DM.json'
identiPath = './trainData/data_identification.csv'
labelPath = './trainData/emotion.csv'

In [1]:
def loadRawTweet(dataPath):
    '''
    Inputs:
        dataPath:  = './trainData/tweets_DM.json'
    Outputs:
         twee_df: data frame aftre sortation
    '''
    with open(dataPath, encoding = 'utf-8-sig') as json_file:    
    #文件不是標準json文件，故要先將文件重新轉回json
    # the file is not standard json file, so I transform the file into json
        data = (line.strip() for line in json_file)    # 因為文件有多行，直接讀取會出錯，因此一行一行讀取  # read by lines   
        tmp = '[{}]'.format(','.join(data))
        twee_df = pd.read_json(tmp)
    
    # 整理'_source'裡的資料: # sorting out the data in "_source"
    info = {i:[] for i in twee_df['_source'][0]['tweet'].keys()}  
    for idx, tweet in enumerate(twee_df['_source']):  
        for key,data in tweet['tweet'].items():
            info[key].append(data)
            
    new = pd.DataFrame(info)
    twee_df = pd.concat([twee_df,new],axis=1)
#    twee_df.drop(columns = ['_source'])
    return twee_df

In [ ]:
def loadData():
    '''
    loading raw data, and make them "DataFame"
    '''
    print('Loading Raw Data...')
    rawData = loadRawTweet(dataPath)
    rawData = rawData.drop(['_source'],axis = 1)
    label = pd.read_csv(labelPath)
    identi = pd.read_csv(identiPath)
    # =============================================================================
    #     spilt into trainning set and testing set
    # =============================================================================
    train_id = identi.loc[identi['identification']=='train']
    test_id  = identi.loc[identi['identification']=='test']
    X_train = pd.DataFrame()
    Y_train = pd.DataFrame()
    X_test = pd.DataFrame()
    Y_test = pd.DataFrame()
    print('Loading training set...')

    tr1 = rawData.tweet_id.isin(train_id['tweet_id'])
    tr2 = label.tweet_id.isin(train_id['tweet_id'])
    X_train = rawData.loc[tr1==True]
    Y_train = label.loc[tr2==True]
    print('Loading testing set...')
    ts1 = rawData.tweet_id.isin(test_id['tweet_id'])
    ts2 = label.tweet_id.isin(test_id['tweet_id'])
    X_test = rawData.loc[ts1==True]
    Y_test = label.loc[ts2==True] 
    
    #--- make sure the ID in X_train and Y_train are same order
    ID = X_train['tweet_id']
    Y_train = pd.merge(ID,Y_train,on='tweet_id')
    
    return X_train, Y_train, X_test

In [ ]:
def label_encode(le,labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le,one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

In [2]:
def eachAccu(pred,gnd,func='f1',labName = {'0':'sadness' ,'1':'disgust' ,'2':'anticipation', '3':'joy' ,'4':'trust' ,'5':'anger', '6':'fear','7':'surprise'}):
    from sklearn.metrics import f1_score
#    pred2 = np.argmax(np.vstack(pred), axis=1).flatten()
    pred3 = np.array(pred)
    gnd = np.array(gnd)
    f1_list = {}
    for key,name in labName.items():
        pred4 = pred3[gnd==int(key)]
        gnd2 = gnd[gnd==int(key)]
        if func == 'f1':
            f1 = f1_score(pred4,gnd2,average='micro')
        f1_list[name] = f1
        print('f1_{}:{:4f}'.format(name,f1))
    return f1_list